# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Part 2 of the assigment

#### Importing Libraries

In [1]:
import numpy as np #Numpy Library
import pandas as pd #Panda Library

In [2]:
from bs4 import BeautifulSoup #BeautifulSoup to read the html

In [3]:
import requests #to request the page

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [5]:
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


#### PART 1 of the assigment

In [6]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
soup = BeautifulSoup(source,'lxml')

In [8]:
tabla = soup.find('table', attrs={'class':'wikitable sortable'})
#print(tabla.prettify())  #prettify to display in a more friendly way

In [9]:
filas = tabla.find_all('tr')
l=[]
for tr in filas:
    td = tr.find_all('td')
    fila = [tr.text for tr in td]
    l.append(fila)
lista = pd.DataFrame(l, columns=['PostCode','Borough','Neigborhood'])

In [10]:
lista = lista.replace('\n','', regex=True)

In [11]:
lista.drop(lista.index[0],inplace=True)

In [12]:
lista = lista[lista.Borough != 'Not assigned']

In [13]:
lista['Neigborhood']= np.where(lista['Neigborhood']=='Not assigned',lista['Borough'],lista['Neigborhood'])

In [14]:
lista.sort_values('PostCode',inplace=True)

In [15]:
concatenar = {
    'Neigborhood': lambda x: ', '.join(x)
}

In [16]:
lista = lista.groupby(['PostCode','Borough'], as_index=False, sort=False).agg(concatenar)

## Part 2 of the assigment

Reading the csv file with the geospatial data for the postal codes that the geocoder returns error

In [17]:
gd=pd.read_csv('https://cocl.us/Geospatial_data',index_col=0)
print('Data loaded')

Data loaded


In [18]:
gd.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [19]:
# The code was removed by Watson Studio for sharing.

Using the geocoder to obtain the latitude and longitude of the Postal Code

In [20]:
lista['Latitude']=0
lista['Longitude']=0
for i in range(0, len(lista)):
    ubicacion ='\''+lista['Borough'].iloc[i]+','+lista['PostCode'].iloc[i]+'\''
    try:
        g=geocoder.here(ubicacion,app_id=ID,app_code=CODE) # I'm using the geocoder HERE
        #print(i,"-",ubicacion,':',g.lat,' ',g.lng)
        lista['Latitude'].iloc[i] = g.lat
        lista['Longitude'].iloc[i]= g.lng
    except: #If the geocoder returns an error, I use gd dataframe to look for the latitude and longitude of the postal code
        postalcode=lista['PostCode'].iloc[i]
        lista['Latitude'].iloc[i] = gd['Latitude'].loc[postalcode]
        lista['Longitude'].iloc[i]= gd['Longitude'].loc[postalcode]
        #print(i,"-",ubicacion,':', lista['Latitude'].iloc[i],' ',lista['Longitude'].iloc[i])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
lista.head(50)

,PostCode,Borough,Neigborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.81153,-79.19552
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.78573,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76569,-79.17526
3,M1G,Scarborough,Woburn,43.76836,-79.21759
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944
5,M1J,Scarborough,Scarborough Village,43.74313,-79.23175
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.72624,-79.26367
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.71313,-79.28505
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.72358,-79.23498
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.69667,-79.26016


In [22]:
lista.shape

(103, 5)

In [23]:
lista.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostCode       103 non-null object
Borough        103 non-null object
Neigborhood    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [24]:
lista.tail(53)

,PostCode,Borough,Neigborhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.681940,-79.378470
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.668160,-79.366600
52,M4Y,Downtown Toronto,Church and Wellesley,43.666590,-79.381300
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657360,-79.378180
55,M5C,Downtown Toronto,St. James Town,43.651210,-79.375480
56,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373670
57,M5G,Downtown Toronto,Central Bay Street,43.656090,-79.384930
58,M5H,Downtown Toronto,"Adelaide, Richmond, King",43.648700,-79.385450
59,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.623470,-79.391510
